# 🛰️ Пример использования окружения GeoSat (Геостационарный спутник)

Этот notebook демонстрирует работу с моделью геостационарного спутника в библиотеке TensorAeroSpace.

## 📋 Что мы изучим:
- Настройка параметров орбитальной симуляции
- Создание и инициализация спутникового окружения
- Выполнение маневров коррекции орбиты
- Анализ динамики спутника


## 📦 Импорт необходимых библиотек

Загружаем все необходимые модули для работы с окружением геостационарного спутника:

In [2]:
# Основные библиотеки
import gymnasium as gym
import numpy as np
from tqdm import tqdm

# Модули TensorAeroSpace для спутниковых систем
from tensoraerospace.envs import GeoSatEnv
from tensoraerospace.utils import generate_time_period, convert_tp_to_sec_tp
from tensoraerospace.signals.standart import unit_step

## ⚙️ Настройка параметров орбитальной симуляции

Определяем основные параметры для моделирования движения геостационарного спутника:

In [3]:
# Параметры временной дискретизации
dt = 0.01  # Шаг дискретизации (секунды)
tp = generate_time_period(tn=20, dt=dt)  # Временной период симуляции (20 секунд)
tps = convert_tp_to_sec_tp(tp, dt=dt)    # Преобразование в секунды

# Параметры орбитальной симуляции
number_time_steps = len(tp)  # Общее количество временных шагов

# Создание эталонного сигнала для коррекции орбиты
# (ступенчатый сигнал 5° на 10-й секунде для маневра)
reference_signals = np.reshape(
    unit_step(degree=5, tp=tp, time_step=10, output_rad=True), 
    [1, -1]
)

print(f'🛰️ Настройки орбитальной симуляции:')
print(f'   • Время симуляции: {tp[-1]:.1f} секунд')
print(f'   • Шаг дискретизации: {dt} секунд')
print(f'   • Количество шагов: {number_time_steps}')
print(f'   • Команда коррекции: {5}° на {10}-й секунде')

## 🌍 Создание и инициализация спутникового окружения

Создаем окружение геостационарного спутника с заданными параметрами:

In [4]:
# Создание окружения геостационарного спутника
env = gym.make(
    'GeoSat-v0',
    number_time_steps=number_time_steps,
    initial_state=[[0], [0], [0]],  # [позиция, скорость, ориентация] - начальное состояние
    output_space=None,              # Автоматическое определение выходного пространства
    reference_signal=reference_signals
)

# Сброс окружения в начальное состояние
initial_observation, info = env.reset()

print(f'🎯 Начальное состояние спутника:')
print(f'   • Состояние 1: {initial_observation[0][0]:.3f}')
print(f'   • Состояние 2: {initial_observation[1][0]:.3f}')
print(f'   • Состояние 3: {initial_observation[2][0]:.3f}')
print(f'   • Размерность состояния: {initial_observation.shape}')

# Возвращаем результат для отображения
(initial_observation, info)

(array([[0.],
        [0.],
        [0.]], dtype=float32),
 {})

## 🚀 Выполнение орбитального маневра

Подаем команду на двигатели коррекции для изменения орбиты спутника:

In [6]:
# Подача команды на двигатели коррекции орбиты
thruster_command = np.array([[1]])  # 1 единица тяги двигателя

# Выполнение одного шага орбитальной симуляции
observation, reward, terminated, truncated, info = env.step(thruster_command)

print(f'🛰️ Состояние после орбитального маневра:')
print(f'   • Состояние 1: {observation[0][0]:.6f}')
print(f'   • Состояние 2: {observation[1][0]:.6f}')
print(f'   • Состояние 3: {observation[2][0]:.6f}')
print(f'
🎯 Результаты маневра:')
print(f'   • Награда: {reward if isinstance(reward, (int, float)) else reward[0]:.8f}')
print(f'   • Миссия завершена: {terminated}')
print(f'   • Симуляция прервана: {truncated}')

## 📡 Анализ телеметрии спутника

Исследуем внутренние данные модели для понимания динамики спутника:

In [9]:
# Анализ истории команд управления
command_history = env.unwrapped.model.store_input

print(f'📈 История команд управления:')
print(f'   • Размерность: {command_history.shape}')
print(f'   • Активные команды: {np.count_nonzero(command_history)}')
print(f'   • Первые 5 команд: {command_history[0][:5]}')
print(f'   • Последние 5 команд: {command_history[0][-5:]}')

# Возвращаем для отображения
command_history

array([[1., 1., 0., ..., 0., 0., 0.]])

In [10]:
# Анализ эталонного сигнала миссии
current_reference = env.unwrapped.reference_signal[0][1]

print(f'🎯 Эталонный сигнал миссии:')
print(f'   • Текущее целевое значение: {current_reference:.3f}')
print(f'   • Размерность сигнала: {env.unwrapped.reference_signal.shape}')
print(f'   • Тип миссии: Коррекция геостационарной орбиты')

# Возвращаем текущее значение
current_reference

0.0

## 🏆 Анализ эффективности миссии

Оцениваем качество выполненного орбитального маневра:

In [11]:
# Детальный анализ эффективности маневра
reward_value = reward[0] if isinstance(reward, np.ndarray) else reward

print(f'🏆 Анализ эффективности миссии:')
print(f'   • Значение награды: {reward_value:.2e}')
print(f'   • Тип данных: {type(reward).__name__}')
print(f'   • Размерность: {reward.shape if hasattr(reward, "shape") else "скаляр"}')

if reward_value > 0:
    print(f'   ✅ Успешный маневр - орбита скорректирована!')
    print(f'   🛰️ Спутник движется к целевой позиции')
elif reward_value < 0:
    print(f'   ⚠️ Неэффективный маневр - требуется коррекция')
    print(f'   🔧 Рекомендуется пересмотреть стратегию управления')
else:
    print(f'   ⚪ Нейтральный результат - спутник в стабильном состоянии')

# Возвращаем значение награды
reward

array([2.34725824e-05])

## 🎉 Заключение

В этом примере мы успешно:

✅ **Создали окружение геостационарного спутника** с правильными параметрами
✅ **Настроили временные параметры** орбитальной симуляции
✅ **Сгенерировали эталонный сигнал** для коррекции орбиты
✅ **Выполнили орбитальные маневры** с помощью двигателей коррекции
✅ **Проанализировали телеметрию** и эффективность миссии

### 🚀 Следующие шаги:
- Попробуйте различные стратегии управления двигателями
- Измените параметры эталонного сигнала для разных типов маневров
- Реализуйте автоматический контроллер для поддержания орбиты
- Визуализируйте траекторию движения спутника
- Добавьте анализ расхода топлива

### 🛰️ Особенности геостационарных спутников:
- **Высота орбиты**: ~35,786 км над экватором
- **Период обращения**: 24 часа (синхронно с вращением Земли)
- **Применение**: Телекоммуникации, метеорология, навигация
- **Коррекция орбиты**: Необходима для компенсации возмущений

---

*Этот notebook демонстрирует возможности TensorAeroSpace для моделирования спутниковых систем и орбитальной механики.*